In [4]:
from langchain_core.tools import tool
import requests

In [37]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.tools import InjectedToolArg
from typing import Annotated

In [ ]:
@tool
def get_conversion_factor(base_currency: str,target_currency: str) ->float:
    """This function fetches the currency conversion factor between a given base currency and a target currency"""
    url = f'https://v6.exchangerate-api.com/v6/433f7b3e5c80da2b4e8dd706/pair/{base_currency}/{target_currency}'
    
    response=requests.get(url)

    return response.json()

@tool
def convert(base_currency_value:int, conversion_rate: Annotated[float,InjectedToolArg]) -> float:
    """Given a currency conversion rate this function calculates the target currency value from a given base currency value"""

    return base_currency_value*conversion_rate

In [27]:
get_conversion_factor.invoke({"base_currency":"USD","target_currency":"INR"})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1746748802,
 'time_last_update_utc': 'Fri, 09 May 2025 00:00:02 +0000',
 'time_next_update_unix': 1746835202,
 'time_next_update_utc': 'Sat, 10 May 2025 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.6035}

In [33]:
convert.invoke({"base_currency_value":10,"conversion_rate":85.3415})

853.415

In [ ]:
## tool binding

llm=ChatOpenAI()

In [ ]:
llm_with_tools=llm.bind_tools([get_conversion_factor,convert])

In [36]:
messages=[HumanMessage("What is the conversion factor between USD and INR and based on that can you convert 10 usd to inr ")]
messages

[HumanMessage(content='What is the conversion factor between USD and INR and based on that can you convert 10 usd to inr ', additional_kwargs={}, response_metadata={})]

In [ ]:
ai_messages=llm_with_tools.invoke(messages)
ai_messages

In [ ]:
messages.append(ai_messages)

In [ ]:
ai_message.tool_calls

In [39]:
import json

for tool_call in ai_messages.tool_calls:
    
    ## Excute the first tool and get the value of conversion rate value
    
    if tool_call["name"]=="get_conversion_factor":
        
        tool_message1=get_conversion_factor.invoke(tool_call)
        
        ## Fetch this conversion rate
        conversion_rate=json.loads(tool_message1.content)['conversion_rate']
        
        ## Append this tool message to message list
        messages.append(tool_messages1)

    ## Excute the 2nd tool using conversion rate from tool1

    if tool_call['name']=="convert":
        ## fetch the current arg
        
        tool_call['args']['conversion_rate']=conversion_rate

        ## Excute the convert tool

        tool_message2=convert.invoke(tool_call)
                
        ## Append this tool message to message list
        
        messages.append(tool_messages1)



NameError: name 'ai_messages' is not defined

In [ ]:
messages

In [ ]:
llm_with_tools.invoke(messages).content